To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4


### Unsloth

In [ ]:
from datasets import load_dataset

dataset = load_dataset("open-r1/OpenR1-Math-220k")

print(dataset)

# 查看第一个训练样本
sample = dataset['train'][0]
print("\n--- Problem ---")
print(sample['problem'])
print("\n--- Solution (The Chain-of-Thought) ---")
print(sample['solution'])
print("\n--- Answer ---")
print(sample['answer'])

def format_function(sample):
    """
    将原始数据样本转换为用于微调的 prompt-completion 对。
    """
    prompt_text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert mathematician. Solve the following problem. Think step by step. End your response with a final answer in the format \\boxed{{answer}}.<|eot_id|>
<|start_header_id|>user<|eot_id|>
{sample['problem']}<|eot_id|>
<|start_header_id|>assistant<|eot_id|>
"""
    
    solution_text = sample['solution'].strip()
    final_answer = sample['answer'].strip()
    
    if f"\\boxed{{{final_answer}}}" not in solution_text:
        completion_text = f"{solution_text}\n\\boxed{{{final_answer}}}"
    else:
        completion_text = solution_text

    return {"prompt": prompt_text, "completion": completion_text}

formatted_dataset = dataset.map(format_function)

train_data = formatted_dataset['train']

train_data.to_json("openr1_math_cot_formatted.jsonl", orient="records", lines=True)

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "glm-4-9b-chat-hf",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0,
)

dataset = load_dataset("json", data_files={"train": "openr1_math_cot_formatted.jsonl"}, split="train")

def preprocess_function(examples):
    """
    在训练前将prompt和completion拼接成一个完整的'text'字段。
    """
    texts = [p + c for p, c in zip(examples['prompt'], examples['completion'])]
    return {"text": texts}

dataset = dataset.map(preprocess_function, batched=True)

args = TrainingArguments(
    output_dir = "glm4-openr1-math",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-5,
    num_train_epochs = 2,
    logging_steps = 50,
    save_strategy = "steps",
    save_steps = 1000,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    max_grad_norm = 1.0,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    args = args,
)

trainer.train()

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
